In [77]:
from info import *
import multiprocessing as mp
import time

In [78]:
show_pro = True     # 是否展示测试过程
trans_exe = True    # 终端输入

In [79]:
z_def = +36  # 根据类型修改
###
test_N = 5 * 10**5
M_sup = 2900

not_flag = True     # 是否通常波

scene = 1           # 0前院 1后院 2屋顶
boom_type = 1       # 0上炸下 1下炸上 2正炸 
boom_col = 6        # 被炸列数
boom_ngt = True    # 是否炸南瓜

jack_type = 0    #Jack参数: 0无限制,1锁早爆,2锁晚爆
plant_list = [  # 添加输出植物格式: [植物所在列,植物类型,是否永动True/False],
    [6,0,True],
    [4,1,True],
]

ice_t = [551]       # 冰时机列表
splash_t = []         # 溅射时机列表
melon = 0           # 固定溅射数量

ash_time = 3000 
ash_type_card = True
vulnerable_time = 0 # 可受伤时机
###

In [80]:
def show_info():
    print(
        test_N,
        M_sup,
        not_flag,
        scene,
        boom_type,
        boom_col,
        boom_ngt,
        jack_type,
        plant_list,
        ice_t,
        splash_t,
        melon,
        ash_time,
        ash_type_card,
        vulnerable_time,
    )
    
if trans_exe:
    #show_info()
    file_name = 'jack_test_info.xlsx'
    info_read = pd.read_excel(file_name, usecols='B', skiprows=0, nrows=6,header=None)
    info_list = info_read.iloc[:,0].tolist()

    tmp = info_list[0]
    if tmp == "全部":   jack_type = 0
    elif tmp == "早爆": jack_type = 1
    else:               jack_type = 2

    tmp = info_list[1]
    if tmp == "前院":     scene = 0
    elif tmp == "后院":   scene = 1
    else:                 scene = 2

    tmp = info_list[2]
    if tmp == "上炸下":     boom_type = 0
    elif tmp == "下炸上":   boom_type = 1
    else:                  boom_type = 2
    boom_col = int(info_list[3])
    boom_ngt = info_list[4] == "南瓜"
    not_flag = info_list[5] =="通常波"

    plant_list = []
    info_read = pd.read_excel(file_name, usecols='D:G', skiprows=2,header=None,na_values='')
    info_read.dropna(inplace=True)
    info_list = info_read.apply(lambda row: row.tolist(), axis=1).tolist()
    for tmp in info_list:
        for _ in range(int(tmp[3])):
            a = int(tmp[0])
            b = 0 if tmp[1]=="曾" else 1
            c = tmp[2]=="永动"
            plant_list.append([a,b,c])

    ice_t = []
    info_read = pd.read_excel(file_name, usecols='I', skiprows=2,header=None,na_values='')
    info_read.dropna(inplace=True)
    info_list = info_read.iloc[:,0].tolist()
    for info in info_list:
        ice_t.append(int(info))

    splash_t = []
    info_read = pd.read_excel(file_name, usecols='K', skiprows=2,header=None,na_values='')
    info_read.dropna(inplace=True)
    info_list = info_read.iloc[:,0].tolist()
    for info in info_list:
        splash_t.append(int(info))

    info_read = pd.read_excel(file_name, usecols='B', skiprows=7, nrows=4,header=None)
    info_list = info_read.iloc[:,0].tolist()
    melon = int(info_list[0])
    test_N = int(info_list[2])  # a * 10**n
    show_pro = info_list[3] == "是"

    info_read = pd.read_excel(file_name, usecols='B', skiprows=12, nrows=4,header=None)
    info_list = info_read.iloc[:,0].tolist()
    ash_time = int(info_list[0])
    M_sup = int(info_list[1])
    ash_type_card = info_list[2] == "卡"
    vulnerable_time = int(info_list[3])

    #show_info()


In [81]:
def get_x_boom():
    if scene == 0:
        return [36,54,70]
    elif scene == 1:
        return [51,62,70]
    elif scene == 2:
        if boom_col >=6:
            return [51,62,70]
        elif boom_col == 5:
            return [29,68,70]
        else:
            if boom_ngt:    return [12,70,69]
            else:           return [23,70,69]

x_target = boom_col*80-10+(30 if boom_ngt else 0) + get_x_boom()[boom_type]
t_ice_atk = np.array(ice_t,dtype=np.int16)
t_splash_atk = np.array(splash_t,dtype=np.int16) + 1

In [82]:

def main_simu(N):
    
    def count_in_range(sorted_array, a, b):
        # [a, b]
        if a > b:  return 0
        if len(sorted_array) == 0:  return 0
        if len(sorted_array) == 1:
            return 1 if (a <= sorted_array[0] and sorted_array[0] <= b) else 0
        if sorted_array[0] > sorted_array[-1]:
            sorted_array = sorted_array[::-1]
        left_idx = np.searchsorted(sorted_array, a, side='left')
        right_idx = np.searchsorted(sorted_array, b, side='right')
        return right_idx - left_idx

    if N == 0: return 0
    # O(NM)
    x, explode_t = generate_animation_x(
        N, M_sup, 
        *ANIMATION_ZOMBIE_INFO['jack'].get_info(),
        not_flag=not_flag, ice_t=ice_t, splash_t=splash_t,
        is_jack=True, jack_type=jack_type
    )
    plants = [IO(
            p[0],p[1],p[2],
            N, M_sup,
            z_def=z_def
        ) for p in plant_list]
    x_explode = x[np.arange(N),explode_t-1].astype(int)
    mask_valid_explosion = (x_explode <= x_target) & (explode_t +110 < ash_time + (0 if ash_type_card else 1)) & (explode_t >= vulnerable_time)

    # O(Nlog(M))
    cnt = 0
    x_int = x.astype(np.int16)
    for idx in np.where(mask_valid_explosion)[0]:
        t_explode = explode_t[idx]
        hp = 335 - 26* melon
        
        cnt_ice_atk = count_in_range(t_ice_atk, 1, t_explode)
        cnt_splash_atk = count_in_range(t_splash_atk, 1, t_explode)
        hp = hp - cnt_ice_atk * 20 - cnt_splash_atk * 26
        
        for p in plants:
            t_atkable = np.searchsorted(-x_int[idx], -p.x_atk, side='left') + 1 - (1 if p.type == 'melon' else 0)
            if p.idle:
                t_trigable = np.searchsorted(-x_int[idx], -p.x_trigger, side='left') + 1
                t_first_trig = p.t_trigger[idx][np.searchsorted(p.t_trigger[idx], t_trigable, side='left')]
                t_atkable = max(t_first_trig, t_atkable)
            cnt_p_atk = count_in_range(p.t_atk[idx], t_atkable, t_explode - (1 if p.type == 'melon' else 0))
            hp = hp - cnt_p_atk * p.dmg
                
        if hp > 0:
            cnt += 1
            
    return cnt


In [83]:
if __name__ == "__main__":    
    inner_N_const = 1* 10**4
    outer_N = test_N//inner_N_const
    inner_Ns = [inner_N_const]*outer_N 
    if test_N%inner_N_const != 0:
        inner_Ns.append(test_N%inner_N_const)  
        outer_N += 1

    start_time = time.time()
    cumulative_res = 0
    cumulative_tests = 0
    # 单线程
    for i,inner_N in enumerate(inner_Ns):
        res = main_simu(inner_N)
        cumulative_res += res
        cumulative_tests += inner_N
        res_current = cumulative_res / cumulative_tests
        if show_pro:    print(f"进度{i+1}/{outer_N}: {res_current*100:,}%")
    end_time = time.time()
    print(f"\n总运行时间: {end_time-start_time:.2f}s")
    print(f"最终结果: {res_current*100:.6f}%")

进度1/1: 0.02%

总运行时间: 1.33s
最终结果: 0.020000%


In [84]:
# 30倍